In [ ]:
from tools import fetch_arxiv_papers

# 1. Tìm kiếm và tải thông tin bài báo
# Hàm này lên Arxiv tìm các bài có từ khóa "Language Models" (lấy 10 bài mới nhất)
papers = fetch_arxiv_papers("Language Models", 10)

In [ ]:
# 2. Kiểm tra nhanh danh sách tiêu đề
# In ra list tiêu đề của các bài vừa tải để xem có đúng chủ đề không
[paper["title"] for paper in papers]

['Evaluation of Oncotimia: An LLM based system for supporting tumour boards',
 'Post-LayerNorm Is Back: Stable, ExpressivE, and Deep',
 'Reflective Translation: Improving Low-Resource Machine Translation via Structured Self-Reflection',
 'EgoHandICL: Egocentric 3D Hand Reconstruction with In-Context Learning',
 'Identifying and Transferring Reasoning-Critical Neurons: Improving LLM Inference Reliability via Activation Steering',
 'HARMONI: Multimodal Personalization of Multi-User Human-Robot Interactions with LLMs',
 'Visual Generation Unlocks Human-Like Reasoning through Multimodal World Models',
 'Neural Neural Scaling Laws',
 'When Iterative RAG Beats Ideal Evidence: A Diagnostic Study in Scientific Multi-hop Question Answering',
 'Zero-Shot Stance Detection in the Wild: Dynamic Target Generation and Multi-Target Adaptation']

In [ ]:
from llama_index.core import Document

# 3. Hàm chuyển đổi dữ liệu thô sang Document
# Convert từ điển (dict) chứa thông tin bài báo sang đối tượng 'Document' của LlamaIndex
# để chuẩn bị cho việc nhúng (embedding)
def create_documents_from_papers(papers):
    documents = []
    for paper in papers:
        # Gom các thông tin quan trọng vào một chuỗi văn bản duy nhất
        content = (
            f"Title: {paper['title']}\n"
            f"Authors: {', '.join(paper['authors'])}\n"
            f"Abstract: {paper['summary']}\n"
            f"URL: {paper.get('pdf_url', 'N/A')}\n"
            f"DOI: {paper.get('doi', 'N/A')}\n"
            f"Primary Category: {paper['primary_category']}\n"
            f"arXiv URL: {paper['arxiv_url']}\n"
        )
        documents.append(Document(text=content))
    return documents

In [ ]:
# 4. Thực hiện chuyển đổi
documents = create_documents_from_papers(papers)

In [ ]:
# 5. Xem thử dữ liệu Document
# In ra danh sách các document đã tạo để kiểm tra trước khi Index
documents

[Document(id_='219fc049-ca88-433a-b064-f0e518fa4dff', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Title: Evaluation of Oncotimia: An LLM based system for supporting tumour boards\nAuthors: Luis Lorenzo, Marcos Montana-Mendez, Sergio Figueiras, Miguel Boubeta, Cristobal Bernardo-Castineira\nAbstract: Multidisciplinary tumour boards (MDTBs) play a central role in oncology decision-making but require manual processes and structuring large volumes of heterogeneous clinical information, resulting in a substantial documentation burden. In this work, we present ONCOTIMIA, a modular and secure clinical tool designed to integrate generative artificial intelligence (GenAI) into oncology workflows and evaluate its application to the automatic completion of lung cancer tumour board forms using large language mo

In [ ]:
from llama_index.core import Settings, VectorStoreIndex 
import constants
import importlib
# Reload constants để đảm bảo nhận config mới nhất (tránh lỗi cache notebook)
importlib.reload(constants)
from constants import embed_model 

# 6. Cấu hình cắt nhỏ dữ liệu (Chunking)
# Chia văn bản thành các đoạn nhỏ 1024 token, gối đầu nhau 50 token
Settings.chunk_size = 1024
Settings.chunk_overlap = 50

# In ra tên model để kiểm tra chắc chắn
print(f"Đang dùng embed model: {embed_model.model_name}")

# 7. Xây dựng Vector Index (QUAN TRỌNG)
# Bước này sẽ gửi dữ liệu lên Google Gemini Embedding để chuyển thành vector số
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model) 
print("✅ Tạo Index thành công!")

Đang dùng embed model: models/gemini-embedding-001
✅ Tạo Index thành công!


In [ ]:
# 8. Lưu Index xuống ổ cứng
# Lưu vào thư mục "index/" để sau này dùng lại (bên file agent.ipynb) mà không cần build lại
index.storage_context.persist(persist_dir="index/")
print("💾 Đã lưu Index vào thư mục 'index/'")

In [ ]:
# 9. Test thử hỏi đáp (RAG)
# Thử hỏi model một câu dựa trên dữ liệu vừa index để xem nó khôn đến đâu
from constants import llm_model

# Tạo query engine với LLM xịn xò (Gemini 2.5 Flash)
query_engine = index.as_query_engine(llm=llm_model)

# Đặt câu hỏi
response = query_engine.query("What are the main topics discussed in these papers?")
print(f"Câu trả lời:\n{response}")

Câu trả lời:
The papers discuss two distinct topics within the field of natural language processing. One paper focuses on the effectiveness of iterative Retrieval-Augmented Generation (RAG) in scientific multi-hop question answering, comparing its performance against static RAG with ideal evidence and analyzing its failure modes. The other paper explores the use of structured self-reflection to enhance machine translation quality, particularly for low-resource languages, by enabling models to critique and revise their own translations.
